In [19]:
import pandas as pd

total_df=pd.read_csv('/home/irteam/junghye-dcloud-dir/pathfinder/data/annotations/annotations_merged.csv')

classes={
    0:'Others',
    1:'Porosity',
    2:'Slag'
}

images_data={}

for i in range(total_df.shape[0]):
    record=total_df.iloc[i]
    temp={
        'file_n':f"./data/Image/{record['dataset']}/{record['image_name']}",
        'width':f"{int(eval(record['img_shape'])[0])}",
        'height':f"{int(eval(record['img_shape'])[1])}",
    }
    file_name=record['image_name'][:-4]
    parts=file_name.split('-')
    prefix = ord(parts[0][0]) - ord('a') + 1
    suffix = parts[1]
    if record['dataset']=='Others':
        image_id = int(f"{prefix}{parts[0][1:]}{suffix}")+1
    elif record['dataset']=='Porosity':
        image_id = int(f"{prefix}{parts[0][1:]}{suffix}")+2
        
    images_data[image_id]=temp
    
print('Image data gathered')
print()


def is_valid(x_min, y_min, x_max, y_max, width, height):
    return (x_min in range(0, width)) and (x_max in range(0, width)) and (y_min in range(0, height)) and (y_max in range(0, width))


images_with_anno=[]
not_valid=0
for i in range(total_df.shape[0]):
    # id 찾기.. 
    record=total_df.iloc[i]
    file_name=record['image_name'][:-4]
    parts=file_name.split('-')
    prefix = ord(parts[0][0]) - ord('a') + 1
    suffix = parts[1]
    if record['dataset']=='Others':
        image_id = int(f"{prefix}{parts[0][1:]}{suffix}")+1
    elif record['dataset']=='Porosity':
        image_id = int(f"{prefix}{parts[0][1:]}{suffix}")+2
        
    img_d=images_data[image_id]
    
    bbox_list=eval(record['bndbox'])
    label_list=eval(record['labels'])
    
    for bbox,label in zip(bbox_list,label_list):
        x_min=int(bbox[0])
        y_min=int(bbox[1])
        x_max=int(bbox[2])
        y_max=int(bbox[3])
        valid=is_valid(x_min,y_min,x_max,y_max,int(img_d['width']),int(img_d['height']))
        
        temp={
            'image_name':img_d['file_n'],
            'x_min':int(bbox[0]),
            'y_min':int(bbox[1]),
            'x_max':int(bbox[2]),
            'y_max':int(bbox[3]),
            'class_name': classes[int(label)]
        }
        if(valid):
            images_with_anno.append(temp)
        else:
            not_valid+=1
            
print("Complete dict generated", "\nFound not valid: ", not_valid)
print()
        
    


Image data gathered

Complete dict generated 
Found not valid:  1



In [20]:
images_with_anno

[{'image_name': './data/Image/Others/j100-1.jpg',
  'x_min': 479,
  'y_min': 262,
  'x_max': 582,
  'y_max': 285,
  'class_name': 'Slag'},
 {'image_name': './data/Image/Others/j100-2.jpg',
  'x_min': 2334,
  'y_min': 186,
  'x_max': 2446,
  'y_max': 213,
  'class_name': 'Slag'},
 {'image_name': './data/Image/Others/j100-3.jpg',
  'x_min': 462,
  'y_min': 185,
  'x_max': 589,
  'y_max': 213,
  'class_name': 'Slag'},
 {'image_name': './data/Image/Others/j101-1.jpg',
  'x_min': 1325,
  'y_min': 212,
  'x_max': 2356,
  'y_max': 305,
  'class_name': 'Slag'},
 {'image_name': './data/Image/Others/j101-1.jpg',
  'x_min': 2529,
  'y_min': 331,
  'x_max': 2561,
  'y_max': 356,
  'class_name': 'Porosity'},
 {'image_name': './data/Image/Others/j101-2.jpg',
  'x_min': 335,
  'y_min': 113,
  'x_max': 369,
  'y_max': 138,
  'class_name': 'Porosity'},
 {'image_name': './data/Image/Others/j101-2.jpg',
  'x_min': 915,
  'y_min': 271,
  'x_max': 1062,
  'y_max': 300,
  'class_name': 'Slag'},
 {'image_nam

In [21]:
df = pd.DataFrame(images_with_anno)

In [22]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.1)

In [23]:
train_df.to_csv('./data/annotations/new_train.csv', index=False, header=None)
test_df.to_csv('./data/annotations/new_test.csv', index=False, header=None)

In [24]:
with open('./data/annotations/classes.csv', 'w') as f:
  for i, (class_name) in enumerate(train_df['class_name'].unique()):
    f.write(f'{class_name},{i}\n')